In [1]:
import os
import openai
import pandas as pd
import numpy as np
import tiktoken
from dotenv import load_dotenv
from openai import OpenAI
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    f1_score,
    precision_score
)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#### Necessary pip installation commands include:
- pip install os
- pip install openai
- pip install pandas
- pip install numpy
- pip install python-dotenv


#### <b>Note</b>: have a <b>.env</b> file already created for accessing API key

In [2]:
# retrieving API Key from OpenAI platform
load_dotenv()
#tokenizer for GPT-3.5 Turbo
tokenizer = tiktoken.get_encoding("cl100k_base")
input_token_cost = ((2.50 / 1000000))
output_token_cost = ((10.00 / 1000000))
FINE_TUNED_TOXIC_DETECTION_API_KEY = os.getenv("FINE_TUNED_TOXIC_DETECTION_API_KEY")
REGULAR_TOXIC_DETECTION_API_KEY = os.getenv("REGULAR_TOXIC_DETECTION_API_KEY")

In [3]:
# instantiating client with API key
general = OpenAI(
    api_key=REGULAR_TOXIC_DETECTION_API_KEY
)

In [4]:
# retrieving dataset utilized for evaluation
df = pd.read_csv('../../data/csv/toxic_classification_evaluation_set.csv')
df['Toxic'] = df['Toxic'].apply(lambda x: str(np.int64(x)))
df

,Text,Toxic
0,believe section Polynomials Maximal LFSRs remo...,0
1,best source Dzierzon biography L Brożek Jan Dz...,0
2,First cover current run Batista cover second i...,0
3,Reviewer Hello account granted reviewer userri...,0
4,Either wikipedia letting anyone become adminis...,1
...,...,...
995,didnt say anything fight Niteshift need put ch...,1
996,P4k Wikipedia delete things people worked worl...,1
997,Shii act intimidation concept intimidation mea...,0
998,would also like say masturbate also gay lol,1


In [5]:
# creating container for holding model completions for comparison
completions = pd.DataFrame(columns=['Text', 'Toxic'])
tokens = pd.DataFrame(columns=['Prompt', 'Input', 'Output', 'Total'])

In [6]:
# processing original GPT-4o model across entire dataset
for index, row in df.iterrows():
    completion = general.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system", 
                "content": "Your job is to detect toxicity from a non-biased viewpoint and label prompted content as either 1 for toxic or 0 for non-toxic"
            },
            {
                "role": "user",
                "content": row['Text']
            }
        ]
    )
    completions.loc[index] = [row['Text'], completion.choices[0].message.content]
    tokens_object = {
        'prompt': len(tokenizer.encode("Your job is to detect toxicity from a non-biased viewpoint and label prompted content as either 1 for toxic or 0 for non-toxic")),
        'input': len(tokenizer.encode(f"Your job is to detect toxicity from a non-biased viewpoint and label prompted content as either 1 for toxic or 0 for non-toxic. {row['Text']}")),
        'output': len(tokenizer.encode(completion.choices[0].message.content))
    }
    total_tokens = tokens_object['input'] + tokens_object['output']
    tokens.loc[index] = [tokens_object['prompt'], tokens_object['input'], tokens_object['output'], total_tokens]


In [7]:
tokens

,Prompt,Input,Output,Total
0,30,67,1,68
1,30,83,1,84
2,30,63,1,64
3,30,137,1,138
4,30,45,1,46
...,...,...,...,...
995,30,55,1,56
996,30,65,1,66
997,30,109,1,110
998,30,40,1,41


In [8]:
completions

,Text,Toxic
0,believe section Polynomials Maximal LFSRs remo...,0
1,best source Dzierzon biography L Brożek Jan Dz...,0
2,First cover current run Batista cover second i...,0
3,Reviewer Hello account granted reviewer userri...,0
4,Either wikipedia letting anyone become adminis...,0
...,...,...
995,didnt say anything fight Niteshift need put ch...,1
996,P4k Wikipedia delete things people worked worl...,0
997,Shii act intimidation concept intimidation mea...,0
998,would also like say masturbate also gay lol,0


In [9]:
# comparing results of model to dataset
compare = completions['Toxic'] == df['Toxic']
gptfour_original_accuracy = compare.values.sum() / compare.size

In [10]:
compare

0       True
1       True
2       True
3       True
4      False
       ...  
995     True
996    False
997     True
998    False
999     True
Name: Toxic, Length: 1000, dtype: bool

In [11]:
# GPT-4o model accuracy with role prompting technique
print(f"Accuracy: {gptfour_original_accuracy * 100:.2f}%")

Accuracy: 80.70%


In [12]:
# total token input cost: $0.16345
(tokens['Input'].sum() * input_token_cost)

0.16345

In [13]:
# total token output cost: $0.016120000000000002
(tokens['Output'].sum() * output_token_cost)

0.016120000000000002

In [14]:
# complete total token cost: $0.17957
(tokens['Input'].sum() * input_token_cost) + (tokens['Output'].sum() * output_token_cost)

0.17957

In [15]:
# 66992
tokens['Total'].sum()

66992